# Delaunay-Watershed for mesh creation: cell aggregate example

In this notebook, we present how to compress segmentation masks of cells in an aggregate into a multimaterial mesh.
We show that one can recompute the mask images from the mesh and seeds for segmentation algorithms if necessary.

## 1) Compute multimaterial mesh with Delaunay Watershed

In [5]:
from dw3d import geometry_reconstruction_3d, reconstruct_mask_from_dict
import skimage.io as io 

def make_idx(n): 
    return(("000"+str(n))[-3:])

dist = 5

image_path = "../data/Images/1.tif"
image = io.imread(image_path)
DW = geometry_reconstruction_3d(image,min_dist = dist, expansion_labels =0,mode='torch',print_info=True)

Mode == Torch
Kernel size = 5
Computing EDT ...
EDT computed in  0.58
Searching local extremas ...
Number of local minimas : 2617
Number of local maxes : 98
Local minimas computed in  2.11
Starting triangulation..
Triangulation build in  0.04
Graph build in  0.681
Watershed done in  0.245


## 2) Reconstruct cell segmentation masks. Visualize in napari

In [3]:
#DW.plot_cells_polyscope()
v = DW.plot_in_napari(add_mesh=True)

/Users/sachaichbiah/opt/anaconda3/lib/python3.8/site-packages/numpy/core/numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
/Users/sachaichbiah/opt/anaconda3/lib/python3.8/site-packages/numpy/core/numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [6]:
DW.return_mesh()

(array([[  0.,   0.,   0.],
        [  0.,   0., 156.],
        [  0., 199.,   0.],
        ...,
        [142., 119.,  72.],
        [142., 120.,  82.],
        [142., 121.,  79.]]),
 array([[ 106,  116,  114,    0,    1],
        [ 106,  114,  118,    0,    1],
        [ 106,  119,  116,    0,    1],
        ...,
        [2718, 2719, 2720,    0,    1],
        [2719, 2722, 2720,    0,    1],
        [2719, 2721, 2722,    0,    1]]))

In [4]:
DW.export_segmentation("Dict_mask.npy")

In [5]:
labels = reconstruct_mask_from_dict("Dict_mask.npy")

In [6]:
import napari
v = napari.view_image(labels)
v.add_image(DW.labels)

<Image layer 'Image' at 0x7f95081497f0>

## 3) Compare files sizes: mesh VS masks 

In [7]:
import sys
import numpy as np 
D = np.load("Dict_mask.npy",allow_pickle=True).item()
Dict_memory_footprint = D['Verts'].nbytes + D['Faces'].nbytes + D['seeds'].nbytes + D['image_shape'].nbytes
print("Size of Delaunay-Watershed data:",Dict_memory_footprint,"bytes")
print("Size of the labels:",labels.nbytes,"bytes")
print("Relative size:", Dict_memory_footprint/labels.nbytes)

Size of Delaunay-Watershed data: 273968 bytes
Size of the labels: 19719200 bytes
Relative size: 0.013893464237900117
